### 1. MongoDB에 웹스크래핑 데이터 저장하기
* MongoDB에 연결
* database와 collection 생성
* actors.json 읽어서 insert_many()

In [2]:
import pymongo
import json

con = pymongo.MongoClient(host='localhost',port=27017)
print(con)

db = con.member_db
print(db)

#collection drop 
db.drop_collection('actors')

col = db.actors
print(col)

with open('data/actors2.json','r',encoding='utf8') as file:
    json_data = json.load(file)
    col.insert_many(json_data)

print('document 건수', col.estimated_document_count())    

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)
Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'member_db')
Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'member_db'), 'actors')
document 건수 600


In [19]:
#컬렉션의 find()를 사용한 함수 정의
def print_find_document(criteria, projection):
    for idx, member_doc in enumerate(col.find(criteria, projection),1):
        print(idx, member_doc)

In [20]:
#컬렉션의 aggregate()를 사용한 함수 정의
def print_aggregate_document(*args):
    agg_list = []
    for arg in args:
        agg_list.append(arg)
    print(agg_list)
    
    for agg_doc in col.aggregate(agg_list):
        print(agg_doc)

In [21]:
#1. 랭킹 1 ~ 50 인 배우의 배우이름,랭킹,흥행지수,소속사,출연영화
criteria = {
    '랭킹':{'$lte':50}
}
projection = {
    '_id':0,'랭킹':1,'배우이름':1,'흥행지수':1,'소속사':1,'출연영화':1
}

print_find_document(criteria, projection)

1 {'배우이름': '성동일', '랭킹': 1, '흥행지수': 137320, '출연영화': ['담보', '방법', '어쩌다 가족']}
2 {'배우이름': '하지원', '랭킹': 2, '흥행지수': 102990, '출연영화': ['담보', '초콜릿', '갈릴레오 : 깨어난 우주']}
3 {'배우이름': '김희원', '랭킹': 3, '흥행지수': 79686, '소속사': 'DMS커뮤니케이션즈', '출연영화': ['담보', '이웃사촌', '바퀴 달린 집']}
4 {'배우이름': '유아인', '랭킹': 4, '흥행지수': 41987, '출연영화': ['소리도 없이', '＃살아있다', '도올아인 오방간다']}
5 {'배우이름': '박소이', '랭킹': 5, '흥행지수': 34330, '출연영화': ['호텔 레이크', '담보', '다시, 봄']}
6 {'배우이름': '유재명', '랭킹': 6, '흥행지수': 27976, '출연영화': ['이태원 클라쓰', '소리도 없이', '소방관']}
7 {'배우이름': '나문희', '랭킹': 7, '흥행지수': 25821, '출연영화': ['오! 문희', '영웅', '감쪽같은 그녀']}
8 {'배우이름': '이희준', '랭킹': 8, '흥행지수': 20714, '출연영화': ['오! 문희', '병훈의 하루', '병훈의 하루']}
9 {'배우이름': '김호중', '랭킹': 9, '흥행지수': 20277, '출연영화': ['그대, 고맙소 : 김호중 생애 첫 팬미팅 무비', '김창옥쇼', '위대한 배태랑']}
10 {'배우이름': '고아성', '랭킹': 10, '흥행지수': 20163, '소속사': '아티스트컴퍼니', '출연영화': ['바닷길 선발대', '항거:유관순 이야기', '삼진그룹 영어토익반']}
11 {'배우이름': '이솜', '랭킹': 11, '흥행지수': 18610, '소속사': '아티스트컴퍼니', '출연영화': ['울렁울렁 울렁대는 가슴안고', '삼진그룹 영어토익반', '구해줘 시즌 2']}
12 {'배우이름': '박혜수',

In [22]:
for actor in col.find({'흥행지수':{'$gte':20000}},{'_id':0,'배우이름':1,'흥행지수':1})\
.sort('배우이름',pymongo.DESCENDING):
    print(actor)

{'배우이름': '하지원', '흥행지수': 102990}
{'배우이름': '이희준', '흥행지수': 20714}
{'배우이름': '유재명', '흥행지수': 27976}
{'배우이름': '유아인', '흥행지수': 41987}
{'배우이름': '성동일', '흥행지수': 137320}
{'배우이름': '박소이', '흥행지수': 34330}
{'배우이름': '나문희', '흥행지수': 25821}
{'배우이름': '김희원', '흥행지수': 79686}
{'배우이름': '김호중', '흥행지수': 20277}
{'배우이름': '고아성', '흥행지수': 20163}


In [23]:
#$exists연산자를 이용
#특기 필드가 존재하면서 
print(col.count_documents({'특기':{'$exists':True}}))
print(col.count_documents({'특기':{'$exists':False}}))

57
543


In [34]:
#다른이름이 필드가 존재하면서 다른 이름이 방탄소년단 인 배우는?
criteria = {
    '다른이름':{'$exists':True},'다른이름':{'$regex':'방탄소년단'}}
projection = {
    '_id':0,'랭킹':1,'배우이름':1,'흥행지수':1,'출연영화':1,'다른이름':1
}
print_find_document(criteria,projection)

In [26]:
#$nor (not or)
#흥행지수>=15000이상or성별=남
#$or 연산자
print(col.count_documents({'$or':[{'흥행지수':{'$gte':15000}},{'성별':'남'}]}))
#nor 연산자
print(col.count_documents({'$nor':[{'흥행지수':{'$lte':15000}},{'성별':'여'}]}))

325
8


In [29]:
criteria = {'$nor':[{'흥행지수':{'$lte':15000}},{'성별':'여'}]}
print_find_document(criteria,projection)

1 {'배우이름': '성동일', '랭킹': 1, '흥행지수': 137320, '출연영화': ['담보', '방법', '어쩌다 가족']}
2 {'배우이름': '김희원', '랭킹': 3, '흥행지수': 79686, '출연영화': ['담보', '이웃사촌', '바퀴 달린 집']}
3 {'배우이름': '유아인', '랭킹': 4, '흥행지수': 41987, '출연영화': ['소리도 없이', '＃살아있다', '도올아인 오방간다']}
4 {'배우이름': '유재명', '랭킹': 6, '흥행지수': 27976, '출연영화': ['이태원 클라쓰', '소리도 없이', '소방관']}
5 {'배우이름': '이희준', '랭킹': 8, '흥행지수': 20714, '출연영화': ['오! 문희', '병훈의 하루', '병훈의 하루']}
6 {'배우이름': '김호중', '랭킹': 9, '흥행지수': 20277, '출연영화': ['그대, 고맙소 : 김호중 생애 첫 팬미팅 무비', '김창옥쇼', '위대한 배태랑']}
7 {'배우이름': '최원영', '랭킹': 13, '흥행지수': 15596, '출연영화': ['오! 문희', '쌍갑포차', '앨리스']}
8 {'배우이름': '조현철', '랭킹': 14, '흥행지수': 15508, '출연영화': ['이웃사촌', '더러운 돈에 손대지 마라', '영화로운 나날']}


#### List에 포함된 문자열 검색
* $all 연산자 사용하면 순서에 상관없이 값으로 찾을 수  있다.

In [33]:
col.count_documents({'출연영화':'아무도 모른다'})
criteria={'출연영화':'아무도 모른다'}
criteria={'출연영화':{'$all':['아무도 모른다','특별근로감독관 조장풍']}}
print_find_document(criteria,projection)

1 {'배우이름': '류덕환', '랭킹': 484, '흥행지수': 0, '출연영화': ['아무도 모른다', '특별근로감독관 조장풍', '난폭한 기록']}


In [36]:
#### $sortBycount 연산자
sort = {
    '$sortByCount':'$소속사'
}
print_aggregate_document(sort)

[{'$sortByCount': '$소속사'}]
{'_id': None, 'count': 541}
{'_id': '아티스트컴퍼니', 'count': 9}
{'_id': '아바 엔터테인먼트', 'count': 4}
{'_id': '나무액터스', 'count': 4}
{'_id': '호두엔터테인먼트', 'count': 3}
{'_id': '㈜사람엔터테인먼트', 'count': 3}
{'_id': 'SM엔터테인먼트', 'count': 2}
{'_id': '심엔터테인먼트', 'count': 2}
{'_id': '싸이더스 HQ', 'count': 2}
{'_id': '라온아이㈜', 'count': 2}
{'_id': '노아엔터테인먼트', 'count': 2}
{'_id': '빌리지엔터테인먼트', 'count': 2}
{'_id': '씨에스엑터스', 'count': 1}
{'_id': '(주)주머니필름', 'count': 1}
{'_id': '헤븐리스타엔터테인먼트', 'count': 1}
{'_id': '예당엔터테인먼트', 'count': 1}
{'_id': '에이바 앤 엔터테인먼트', 'count': 1}
{'_id': '(주)글로리어스', 'count': 1}
{'_id': '(주)이스트필름', 'count': 1}
{'_id': '㈜팬엔터테인먼트', 'count': 1}
{'_id': '케이엠컬쳐(주)', 'count': 1}
{'_id': '자이온 이엔티(주)', 'count': 1}
{'_id': '비에스스타엔터테인먼트', 'count': 1}
{'_id': '열음엔터테인먼트', 'count': 1}
{'_id': '(주)에프엔씨뮤직', 'count': 1}
{'_id': 'G.G 엔터테인먼트', 'count': 1}
{'_id': '아이디미디어', 'count': 1}
{'_id': 'DMS커뮤니케이션즈', 'count': 1}
{'_id': '지티비엔터테인먼트', 'count': 1}
{'_id': '멘토엔터테인먼트', 'count': 1}
{'_id': '